# RnD-1: симуляции множественных проверок и рерандомизации

In [1]:
import random, math, statistics
random.seed(42); alpha=0.05

In [2]:
def normal_cdf(x): return 0.5*(1+math.erf(x/math.sqrt(2)))
def pvalue(a,b):
    m1,m2=statistics.mean(a),statistics.mean(b)
    v1,v2=statistics.pvariance(a),statistics.pvariance(b)
    se=math.sqrt(v1/len(a)+v2/len(b)+1e-12)
    z=abs(m1-m2)/se
    return 2*(1-normal_cdf(z))
def smd(a,b):
    v=(statistics.pvariance(a)+statistics.pvariance(b))/2
    return 0 if v<=0 else abs((statistics.mean(a)-statistics.mean(b))/math.sqrt(v))

In [3]:
print('Эксперимент 1: рост P(min p < alpha)')
for K in [5,10,20,40]:
    c=0
    for _ in range(12):
        n=160
        g=[random.randint(0,1) for _ in range(n)]
        pvals=[]
        for _ in range(K):
            col=[random.gauss(0,1) for _ in range(n)]
            a=[x for x,t in zip(col,g) if t==0]; b=[x for x,t in zip(col,g) if t==1]
            pvals.append(pvalue(a,b))
        c += (min(pvals)<alpha)
    print(K, round(c/12,3))

Эксперимент 1: рост P(min p < alpha)
5 0.333
10 0.25
20 0.667
40 0.917


In [4]:
print('\nЭксперимент 2: сравнение процедур')
for K in [5,10,20,40]:
    raw=bon=holm=bh=0
    for _ in range(12):
        n=160; g=[random.randint(0,1) for _ in range(n)]
        pvals=[]
        for _ in range(K):
            col=[random.gauss(0,1) for _ in range(n)]
            a=[x for x,t in zip(col,g) if t==0]; b=[x for x,t in zip(col,g) if t==1]
            pvals.append(pvalue(a,b))
        raw += any(p<alpha for p in pvals)
        bon += any(p<alpha/K for p in pvals)
        sp=sorted(pvals)
        holm += any(sp[i] < alpha/(K-i) for i in range(K))
        kmax=-1
        for i,p in enumerate(sp, start=1):
            if p <= alpha*i/K: kmax=i
        bh += (kmax!=-1)
    print(K, {'raw':round(raw/12,3),'bonf':round(bon/12,3),'holm':round(holm/12,3),'bh':round(bh/12,3)})


Эксперимент 2: сравнение процедур
5 {'raw': 0.333, 'bonf': 0.0, 'holm': 0.0, 'bh': 0.0}
10 {'raw': 0.5, 'bonf': 0.167, 'holm': 0.167, 'bh': 0.167}
20 {'raw': 0.667, 'bonf': 0.0, 'holm': 0.0, 'bh': 0.0}
40 {'raw': 1.0, 'bonf': 0.083, 'holm': 0.083, 'bh': 0.083}


In [5]:
print('\nЭксперимент 3: p-gate vs max|SMD| gate')
for K in [10,20,40]:
    p_it=[]; s_it=[]
    for _ in range(12):
        n=180
        cols=[[random.gauss(0,1) for _ in range(n)] for _ in range(K)]
        for i in range(1,120):
            g=[random.randint(0,1) for _ in range(n)]
            pvals=[]
            for col in cols:
                a=[x for x,t in zip(col,g) if t==0]; b=[x for x,t in zip(col,g) if t==1]
                pvals.append(pvalue(a,b))
            if all(p>=0.05 for p in pvals): p_it.append(i); break
        for i in range(1,120):
            g=[random.randint(0,1) for _ in range(n)]
            smds=[]
            for col in cols:
                a=[x for x,t in zip(col,g) if t==0]; b=[x for x,t in zip(col,g) if t==1]
                smds.append(smd(a,b))
            if max(smds)<=0.12: s_it.append(i); break
    mp = statistics.median(p_it) if p_it else None
    ms = statistics.median(s_it) if s_it else None
    print(K, {'median_p_iter':mp, 'median_smd_iter':ms})


Эксперимент 3: p-gate vs max|SMD| gate
10 {'median_p_iter': 1.0, 'median_smd_iter': 85.5}
20 {'median_p_iter': 1.5, 'median_smd_iter': None}
40 {'median_p_iter': 6.0, 'median_smd_iter': None}
